In [6]:
import pandas as pd

In [7]:
# Load both summaries
bucket_df = pd.read_csv('bucket_coil_signal_summary_2std.csv')
no_bucket_df = pd.read_csv('withoutbucket_coil_signal_summary_2std.csv')

In [8]:
#Calculate Outside_Percent columns
bucket_df['Outside_Percent'] = bucket_df['No_of_Coils_outside_range'] / bucket_df['No_of_Coils_in_bucket']
no_bucket_df['Outside_Percent'] = no_bucket_df['No_of_Coils_outside_range'] / no_bucket_df['No_of_Coils']

# Aggregate bucket data by signal (sum of outside ranges / sum of coils)
agg_bucket = bucket_df.groupby('Signal_Name').apply(
    lambda x: x['No_of_Coils_outside_range'].sum() / x['No_of_Coils_in_bucket'].sum()
).reset_index(name='Outside_Percent_Bucket')

# Select relevant columns in no_bucket_df and rename for clarity
no_bucket_agg = no_bucket_df[['Signal_Name', 'Outside_Percent']].rename(columns={'Outside_Percent': 'Outside_Percent_NoBucket'})

# Merge on Signal_Name
comparison_df = pd.merge(
    agg_bucket,
    no_bucket_agg,
    on='Signal_Name',
    how='inner'
)

# Calculate improvement: positive means bucketing lowered outside percentage
comparison_df['Improvement'] = comparison_df['Outside_Percent_NoBucket'] - comparison_df['Outside_Percent_Bucket']

print(comparison_df.sort_values(by='Improvement', ascending=False))

                                         Signal_Name  Outside_Percent_Bucket  \
108                  Tension Reel Calculated Tension                0.025490   
3                Exit Tension Reel Tension Reference                0.023529   
47                       Stand 1-2 Tension Reference                0.027451   
48                  Stand 1-2 Total Tension Feedback                0.041176   
68                       Stand 2-3 Tension Reference                0.023529   
..                                               ...                     ...   
6    Morgoil DriveBottom Bearing Outflow Temp Stand2                0.031373   
12      Morgoil DriveTop Bearing Outflow Temp Stand4                0.031373   
11      Morgoil DriveTop Bearing Outflow Temp Stand3                0.031373   
8    Morgoil DriveBottom Bearing Outflow Temp Stand4                0.031373   
9       Morgoil DriveTop Bearing Outflow Temp Stand1                0.033333   

     Outside_Percent_NoBucket  Improvem

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_23752\1322267924.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_bucket = bucket_df.groupby('Signal_Name').apply(


In [9]:
comparison_df.to_csv("Bucket vs without Bucket.csv",index=False)